# 집중해조
: 2024-2 이화여자대학교 휴먼기계바이오공학부 융합캡스톤

## before fine-tuning

### some lectures test

- random test : arch, comp, elec, huma, lang 5개의 과목에 대해 랜덤으로 5개의 강의 테스트
- selected lecture
    - 'arch\\C06613', 'arch\\C01748', 'arch\\C02799', 'arch\\C04157', arch\\C04006'
    - 'comp\\C08360', 'comp\\C08367', 'comp\\C07447', 'comp\\C12319', 'comp\\C05378'
    - 'elec\\C02824', 'elec\\C06168', 'elec\\C02727', 'elec\\C13269', 'elec\\C02689'
    - 'huma\\C00537', 'huma\\C01442', 'huma\\C13960', 'huma\\C07347', 'huma\\C04142'
    - 'lang\\C03841', 'lang\\C10365', 'lang\\C13586', 'lang\\C05227', 'lang\\C03843'

In [1]:
import os
import glob

def find_all_lecture_files(source_dir, label_dir):
    """
    디렉토리 전체를 탐색하여 모든 강의의 wav 파일과 json 파일을 찾습니다.
    source_dir: wav 파일 경로가 있는 루트 디렉토리
    label_dir: json 파일 경로가 있는 루트 디렉토리
    """
    all_wav_files = []
    all_json_files = []
    
    # 디렉토리 전체 탐색
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith(".wav"):  # WAV 파일 찾기
                # wav 파일 경로
                wav_path = os.path.join(root, file)
                
                # 해당 wav 파일에 대응하는 json 파일 경로 생성
                relative_path = os.path.relpath(wav_path, source_dir)
                json_path = os.path.join(label_dir, relative_path).replace(".wav", ".json")
                
                # json 파일이 존재하면 리스트에 추가
                if os.path.exists(json_path):
                    all_wav_files.append(wav_path)
                    all_json_files.append(json_path)
    
    return all_wav_files, all_json_files

# 루트 디렉토리 경로 설정
source_dir = "test/source"
label_dir = "test/labels"

# 모든 강의 파일 탐색
wav_files, json_files = find_all_lecture_files(source_dir, label_dir)

print("Total WAV Files: ", len(wav_files))
print("Total JSON Files: ", len(json_files))


Total WAV Files:  71652
Total JSON Files:  71652


In [2]:
import os
import json
import random
import torch
from datasets import Dataset
from transformers import pipeline
from jiwer import wer

# 디바이스 설정 (GPU 사용 가능 시 GPU로 설정)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Whisper 모델 로드
pipe = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-medium",
    device=0 if device == "cuda" else -1,
    return_timestamps=True
)

# 랜덤으로 5개의 강의 디렉토리 선택 함수
def get_random_lectures(root_dir, subdirs, num_samples=5):
    """
    각 하위 디렉토리(subdirs)에서 랜덤으로 num_samples만큼 강의를 선택.
    """
    selected_lectures = []
    for subdir in subdirs:
        lecture_path = os.path.join(root_dir, subdir)
        if os.path.exists(lecture_path):
            # 강의 디렉토리들 가져오기
            lectures = [os.path.join(lecture_path, d) for d in os.listdir(lecture_path) if os.path.isdir(os.path.join(lecture_path, d))]
            # 랜덤으로 num_samples만큼 선택
            selected_lectures.extend(random.sample(lectures, min(num_samples, len(lectures))))
    return selected_lectures

# 강의 디렉토리 루트 설정
source_root = "test/source"
label_root = "test/labels"
categories = ["eng/arch", "eng/comp", "eng/elec", "hum/huma", "hum/lang"]

# 랜덤으로 5개의 강의 선택
selected_lectures = get_random_lectures(source_root, categories, num_samples=5)
print("Selected Lectures:", selected_lectures)

# 선택된 강의에서 wav와 json 파일 수집
wav_files = []
json_files = []
for lecture in selected_lectures:
    for wav_file in glob.glob(os.path.join(lecture, "*.wav")):
        # JSON 파일 경로 생성
        relative_path = os.path.relpath(wav_file, source_root)
        json_file = os.path.join(label_root, relative_path).replace(".wav", ".json")
        if os.path.exists(json_file):
            wav_files.append(wav_file)
            json_files.append(json_file)

print(f"Total WAV Files: {len(wav_files)}, Total JSON Files: {len(json_files)}")

# ASR 수행 및 평가: Hugging Face Datasets 활용
if len(wav_files) > 0:
    # 데이터셋 생성
    data = Dataset.from_dict({"audio": wav_files})

    # ASR 수행
    asr_results = pipe(data["audio"])  # Whisper 모델로 일괄 처리

    # 평가 결과 저장
    evaluation_results = []
    for wav_file, json_file, asr_result in zip(wav_files, json_files, asr_results):
        # 라벨 데이터 로드
        with open(json_file, 'r', encoding='utf-8') as f:
            label_data = json.load(f)
            label_text = label_data["06_transcription"]["1_text"]

        # WER 계산
        error_rate = wer(label_text, asr_result["text"])

        # 결과 저장
        evaluation_results.append({
            "wav_file": wav_file,
            "json_file": json_file,
            "asr_text": asr_result["text"],
            "label_text": label_text,
            "wer": error_rate
        })

    # 평균 WER 계산
    average_wer = sum(result["wer"] for result in evaluation_results) / len(evaluation_results)

    # 결과 저장
    output_data = {
        "evaluation_results": evaluation_results,
        "average_wer": average_wer
    }
    output_file = "random_evaluation_results.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)

    print(f"Random evaluation results saved to {output_file}")
else:
    print("No files found for processing.")


Device: cuda
Selected Lectures: ['test/source\\eng/arch\\C06613', 'test/source\\eng/arch\\C01748', 'test/source\\eng/arch\\C02799', 'test/source\\eng/arch\\C04157', 'test/source\\eng/arch\\C04006', 'test/source\\eng/comp\\C08360', 'test/source\\eng/comp\\C08367', 'test/source\\eng/comp\\C07447', 'test/source\\eng/comp\\C12319', 'test/source\\eng/comp\\C05378', 'test/source\\eng/elec\\C02824', 'test/source\\eng/elec\\C06168', 'test/source\\eng/elec\\C02727', 'test/source\\eng/elec\\C13269', 'test/source\\eng/elec\\C02689', 'test/source\\hum/huma\\C00537', 'test/source\\hum/huma\\C01442', 'test/source\\hum/huma\\C13960', 'test/source\\hum/huma\\C07347', 'test/source\\hum/huma\\C04142', 'test/source\\hum/lang\\C03841', 'test/source\\hum/lang\\C10365', 'test/source\\hum/lang\\C13586', 'test/source\\hum/lang\\C05227', 'test/source\\hum/lang\\C03843']


c:\Users\yein\anaconda3\envs\cap\lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Total WAV Files: 7858, Total JSON Files: 7858


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending t

Random evaluation results saved to random_evaluation_results.json


### one lecture test

In [2]:
import os
import glob

# 특정 강의 경로 설정
source_path = "test/source/eng/arch/C01748"
label_path = "test/labels/eng/arch/C01748"

# 모든 wav 파일과 json 파일 검색
wav_files = glob.glob(os.path.join(source_path, "*.wav"))
json_files = glob.glob(os.path.join(label_path, "*.json"))

print("WAV Files (", len(wav_files), "):", wav_files)
print("JSON Files (", len(json_files), "):", json_files)

WAV Files ( 252 ): ['test/source/eng/arch/C01748\\U00000.wav', 'test/source/eng/arch/C01748\\U00001.wav', 'test/source/eng/arch/C01748\\U00002.wav', 'test/source/eng/arch/C01748\\U00003.wav', 'test/source/eng/arch/C01748\\U00004.wav', 'test/source/eng/arch/C01748\\U00005.wav', 'test/source/eng/arch/C01748\\U00006.wav', 'test/source/eng/arch/C01748\\U00007.wav', 'test/source/eng/arch/C01748\\U00008.wav', 'test/source/eng/arch/C01748\\U00009.wav', 'test/source/eng/arch/C01748\\U00010.wav', 'test/source/eng/arch/C01748\\U00011.wav', 'test/source/eng/arch/C01748\\U00012.wav', 'test/source/eng/arch/C01748\\U00013.wav', 'test/source/eng/arch/C01748\\U00014.wav', 'test/source/eng/arch/C01748\\U00015.wav', 'test/source/eng/arch/C01748\\U00016.wav', 'test/source/eng/arch/C01748\\U00017.wav', 'test/source/eng/arch/C01748\\U00018.wav', 'test/source/eng/arch/C01748\\U00019.wav', 'test/source/eng/arch/C01748\\U00020.wav', 'test/source/eng/arch/C01748\\U00021.wav', 'test/source/eng/arch/C01748\\U000

In [3]:
import torch
from transformers import pipeline
import concurrent.futures

# 디바이스 설정 (GPU 사용 가능 시 GPU로 설정)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Whisper medium 모델 로드
pipe = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-medium",
    device=0 if device == "cuda" else -1
)

# ASR 작업을 수행하는 함수
def process_file(wav_file):
    print(f"Processing: {wav_file}")
    result = pipe(wav_file)
    return wav_file, result["text"]

# 모든 wav 파일에 대해 병렬로 ASR 수행
wav_files = wav_files[:50]  # 실제 파일 리스트로 대체해야 합니다
asr_results = {}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_file, wav_file): wav_file for wav_file in wav_files}
    for future in concurrent.futures.as_completed(futures):
        wav_file, text = future.result()
        asr_results[wav_file] = text

# 결과 출력
print(asr_results)

Device: cuda


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing: test/source/eng/arch/C01748\U00000.wav
Processing: test/source/eng/arch/C01748\U00001.wav
Processing: test/source/eng/arch/C01748\U00002.wav
Processing: test/source/eng/arch/C01748\U00003.wav
Processing: test/source/eng/arch/C01748\U00004.wav
Processing: test/source/eng/arch/C01748\U00005.wav
Processing: test/source/eng/arch/C01748\U00006.wav
Processing: test/source/eng/arch/C01748\U00007.wav
Processing: test/source/eng/arch/C01748\U00008.wav
Processing: test/source/eng/arch/C01748\U00009.wav
Processing: test/source/eng/arch/C01748\U00010.wav
Processing: test/source/eng/arch/C01748\U00011.wav
Processing: test/source/eng/arch/C01748\U00012.wav
Processing: test/source/eng/arch/C01748\U00013.wav
Processing: test/source/eng/arch/C01748\U00014.wav
Processing: test/source/eng/arch/C01748\U00015.wav


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
c:\Users\yein\anaconda3\envs\cap\lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Processing: test/source/eng/arch/C01748\U00016.wav
Processing: test/source/eng/arch/C01748\U00017.wav
Processing: test/source/eng/arch/C01748\U00018.wav
Processing: test/source/eng/arch/C01748\U00019.wav
Processing: test/source/eng/arch/C01748\U00020.wav
Processing: test/source/eng/arch/C01748\U00021.wav
Processing: test/source/eng/arch/C01748\U00022.wav
Processing: test/source/eng/arch/C01748\U00023.wav
Processing: test/source/eng/arch/C01748\U00024.wav
Processing: test/source/eng/arch/C01748\U00025.wav
Processing: test/source/eng/arch/C01748\U00026.wav
Processing: test/source/eng/arch/C01748\U00027.wav
Processing: test/source/eng/arch/C01748\U00028.wav
Processing: test/source/eng/arch/C01748\U00029.wav
Processing: test/source/eng/arch/C01748\U00030.wav
Processing: test/source/eng/arch/C01748\U00031.wav
Processing: test/source/eng/arch/C01748\U00032.wav
Processing: test/source/eng/arch/C01748\U00033.wav
Processing: test/source/eng/arch/C01748\U00034.wav
Processing: test/source/eng/arc

In [4]:
import json

# 라벨 데이터를 로드하여 비교
evaluation_results = {}
for wav_file, json_file in zip(wav_files, json_files):
    with open(json_file, 'r', encoding='utf-8') as f:
        label_data = json.load(f)
        label_text = label_data["06_transcription"]["1_text"]
    
    asr_text = asr_results[wav_file]
    evaluation_results[wav_file] = {
        "asr_text": asr_text,
        "label_text": label_text
    }

# 결과 출력
for file, result in evaluation_results.items():
    print(f"\nFile: {file}")
    print(f"ASR: {result['asr_text']}")
    print(f"Label: {result['label_text']}")


File: test/source/eng/arch/C01748\U00000.wav
ASR:  학생 여러분 안녕하십니까
Label: 학생 여러분 안녕하십니까?

File: test/source/eng/arch/C01748\U00001.wav
ASR:  지금부터 정력학 1주 2차 시강이 시작하도록 하겠습니다.
Label: 지금부터 정역학 1주 2차시 강의 시작하도록 하겠습니다.

File: test/source/eng/arch/C01748\U00002.wav
ASR:  이 시간에는 정력학에서 다루게 될 힘의 단위 가장 많이 나오는 게 힘이기 때문에 이 힘의 단위를 먼저 정리하고 그리고 난 다음에 1장부터 진도를 나가도록 하겠습니다.
Label: 이 시간에는 이 정역학에서 다루게 될 힘의 단위 가장 많이 나오는 게 힘이기 때문에 힘의 단위를 한번 정리하고 그러고 난 다음에 1장부터 진도를 나가도록 하겠습니다.

File: test/source/eng/arch/C01748\U00003.wav
ASR:  정력학에서 사용하는 힘의 단위로는 우리가 힘 하면 뉴턴과 킬로그램이 떠오르게 됩니다.
Label: 정역학에서 사용하는 힘의 단위로는 우리가 이제 힘 하면은 이제 뉴턴과 킬로그램이 떠오르게 됩니다.

File: test/source/eng/arch/C01748\U00004.wav
ASR:  여러분들이 이미 중학교 때, 고등학교 때, 뉴턴 이라고 하는 단위를 사용하셨을 거예요.
Label: 여러분들이 이미 중학교 때나 고등학교 때 이 뉴턴이라고 하는 단위를 사용하셨을 거예요.

File: test/source/eng/arch/C01748\U00005.wav
ASR:  그러나 우리가 실생활에서는 유턴을 잘 사용하지 않죠.
Label: 그러나 우리가 실생활에서는 뉴턴을 잘 사용하지 않죠?

File: test/source/eng/arch/C01748\U00006.wav
ASR:  실생활에서는 뭘 씁니까?
Label: 실생활에서는 뭘 씁니까?

File: test/source/

In [5]:
from jiwer import wer

# WER 계산
for file, result in evaluation_results.items():
    asr_text = result["asr_text"]
    label_text = result["label_text"]
    error_rate = wer(label_text, asr_text)
    evaluation_results[file]["wer"] = error_rate

# 결과 출력
for file, result in evaluation_results.items():
    print(f"\nFile: {file}")
    print(f"ASR: {result['asr_text']}")
    print(f"Label: {result['label_text']}")
    print(f"WER: {result['wer']}")


File: test/source/eng/arch/C01748\U00000.wav
ASR:  학생 여러분 안녕하십니까
Label: 학생 여러분 안녕하십니까?
WER: 0.3333333333333333

File: test/source/eng/arch/C01748\U00001.wav
ASR:  지금부터 정력학 1주 2차 시강이 시작하도록 하겠습니다.
Label: 지금부터 정역학 1주 2차시 강의 시작하도록 하겠습니다.
WER: 0.42857142857142855

File: test/source/eng/arch/C01748\U00002.wav
ASR:  이 시간에는 정력학에서 다루게 될 힘의 단위 가장 많이 나오는 게 힘이기 때문에 이 힘의 단위를 먼저 정리하고 그리고 난 다음에 1장부터 진도를 나가도록 하겠습니다.
Label: 이 시간에는 이 정역학에서 다루게 될 힘의 단위 가장 많이 나오는 게 힘이기 때문에 힘의 단위를 한번 정리하고 그러고 난 다음에 1장부터 진도를 나가도록 하겠습니다.
WER: 0.2

File: test/source/eng/arch/C01748\U00003.wav
ASR:  정력학에서 사용하는 힘의 단위로는 우리가 힘 하면 뉴턴과 킬로그램이 떠오르게 됩니다.
Label: 정역학에서 사용하는 힘의 단위로는 우리가 이제 힘 하면은 이제 뉴턴과 킬로그램이 떠오르게 됩니다.
WER: 0.3076923076923077

File: test/source/eng/arch/C01748\U00004.wav
ASR:  여러분들이 이미 중학교 때, 고등학교 때, 뉴턴 이라고 하는 단위를 사용하셨을 거예요.
Label: 여러분들이 이미 중학교 때나 고등학교 때 이 뉴턴이라고 하는 단위를 사용하셨을 거예요.
WER: 0.3333333333333333

File: test/source/eng/arch/C01748\U00005.wav
ASR:  그러나 우리가 실생활에서는 유턴을 잘 사용하지 않죠.
Label: 그러나 우리가 실생활에서는 뉴턴을 잘 사용하지 않죠?


In [6]:
# 전체 WER 평균 계산
total_wer = sum(result["wer"] for result in evaluation_results.values()) / len(evaluation_results)
print(f"Average WER for the lecture: {total_wer}")

Average WER for the lecture: 0.449418506181664
